# Accessing PZ outputs directly from the DP1 butler

## Prerequisites

Need to clone the following repositories (and branch) and add the eups setup commands to include them in your `notebooks/.user_setups` environment:
- [`testdata_rail`](https://github.com/lsst/testdata_rail): `main`
- [`meas_pz`](https://github.com/lsst-dm/meas_pz): `u/echarles/all_the_estimators`

In [ ]:
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/dp1')

In [ ]:
pz_collection = 'LSSTComCam/runs/DRP/u/echarles/pz_redshift_outlet/dp1/gold/baseline/v0'

In [ ]:
butler.registry.queryCollections('LSSTComCam/runs/DRP/u/echarles/pz_redshift_outlet/dp1/gold/baseline*')

In [ ]:
ens = butler.get('pz_estimate_knn', instrument='LSSTComCam', skymap='lsst_cells_v1', tract=5063, collections=pz_collection)

In [ ]:
ens.ancil